In [10]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langchain_experimental langsmith pandas langgraph

In [11]:
%pip install --upgrade --quiet  google-search-results

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("SERP_API_KEY")
_set_if_undefined("SERPAPI_API_KEY")




# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Hierarchical Teams With Supervisors"

## Tools

Research Team Tools

In [2]:
from typing import Annotated, List, Tuple, Union

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain.agents import load_tools

tavily_tool = TavilySearchResults(max_results=5)
google_finance_tools = load_tools(["google-scholar", "google-finance"])


technical_analyst_tools = google_finance_tools + [tavily_tool]

economist_tools = google_finance_tools.copy()

banking_analyst_tools = google_finance_tools.copy()

industrial_sector_analyst_tools = google_finance_tools.copy()

Human Resources Tools

In [3]:
recruitment_specialist_tools = [tavily_tool]

payroll_and_personnel_tools = [tavily_tool]

Help Desk Tools

In [4]:
network_malfunction_specialist_tools = [tavily_tool]

mobile_banking_malfunction_specialist_tools = [tavily_tool]

transaction_error_specialist_tools = [tavily_tool]

customer_satisfaction_specialist_tools = [tavily_tool]

## Helper Utilities

In [5]:
from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph


def create_agent(
    llm: ChatOpenAI,
    tools: list,
    system_prompt: str,
) -> str:
    """Create a function-calling agent and add it to the graph."""
    system_prompt += "\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason! You are one of the following team members: {team_members}."
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


def create_team_supervisor(llm: ChatOpenAI, system_prompt, members) -> str:
    """An LLM-based router."""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options), team_members=", ".join(members))
    return (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )

In [6]:
#Template to prevent endless loops

prevent_endless_loops_template = "Do not call this tool more than once. Do not call another tool if this returns results."

## Research Team

In [7]:
import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import functools

# Research team graph state
class ResearchTeamState(TypedDict):
    # A message is added after each team member finishes
    messages: Annotated[List[BaseMessage], operator.add]
    # The team members are tracked so they are aware of
    # the others' skill-sets
    team_members: List[str]
    # Used to route work. The supervisor calls a function
    # that will update this every time it makes a decision
    next: str

llm = ChatOpenAI(model="gpt-4-1106-preview")

technical_analyst_agent = create_agent(
    llm,
    technical_analyst_tools,
    "You are a Technical Analyst responsible for analyzing technical data and trends within a specific industry or market, providing insights and recommendations based on your analysis." + prevent_endless_loops_template,
)
technical_analyst_node = functools.partial(agent_node, agent=technical_analyst_agent, name="Technical_Analyst")

economist_agent = create_agent(
    llm,
    economist_tools,
    "You are an Economist tasked with studying economic indicators, trends, and policies to assess their impact on businesses or markets, and offering strategic advice or forecasts based on your findings." + prevent_endless_loops_template,
)
economist_node = functools.partial(agent_node, agent=economist_agent, name="Economist")

banking_analyst_agent = create_agent(
    llm,
    banking_analyst_tools,
    "You are a Banking Analyst specialized in analyzing financial data related to banking operations, such as loan portfolios, interest rate trends, or investment strategies, to assist banks in making informed decisions." + prevent_endless_loops_template,
)
banking_analyst_node = functools.partial(agent_node, agent=banking_analyst_agent, name="Banking_Analyst")

industrial_sector_analyst_agent = create_agent(
    llm,
    industrial_sector_analyst_tools,
    "You are an Industrial Sector Analyst focused on researching and evaluating data related to specific industries or sectors, such as manufacturing, energy, or technology, to identify opportunities, risks, and market trends for stakeholders." + prevent_endless_loops_template,
)
industrial_sector_analyst_node = functools.partial(agent_node, agent=industrial_sector_analyst_agent, name="Industrial_Sector_Analyst")


supervisor_agent = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  Technical_Analyst, Economist, Banking_Analyst, Industrial_Sector_Analyst. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH. Do not call a tool more than once."
    "Do not call another tool if the tool returns results.",
    ["Technical_Analyst", "Economist" , "Banking_Analyst" , "Industrial_Sector_Analyst"],
)

Research Team Graph

In [8]:
research_graph = StateGraph(ResearchTeamState)
research_graph.add_node("Technical_Analyst", technical_analyst_node)
research_graph.add_node("Economist", economist_node)
research_graph.add_node("Banking_Analyst", banking_analyst_node)
research_graph.add_node("Industrial_Sector_Analyst", industrial_sector_analyst_node)
research_graph.add_node("supervisor", supervisor_agent)

# Define the control flow
research_graph.add_edge("Technical_Analyst", "supervisor")
research_graph.add_edge("Economist", "supervisor")
research_graph.add_edge("Banking_Analyst", "supervisor")
research_graph.add_edge("Industrial_Sector_Analyst", "supervisor")

research_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Technical_Analyst": "Technical_Analyst",
        "Economist": "Economist",
        "Banking_Analyst": "Banking_Analyst",
        "Industrial_Sector_Analyst": "Industrial_Sector_Analyst",
        "FINISH": END
    },
)


research_graph.set_entry_point("supervisor")
chain = research_graph.compile()


# The following functions interoperate between the top level graph state
# and the state of the research sub-graph
# this makes it so that the states of each graph don't get intermixed
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
    }
    return results


research_chain = enter_chain | chain

Research Team Individual Test

In [9]:
for s in research_chain.stream(
"Government policies regarding tariffs on imported goods are undergoing changes. How might this affect our supply chain and overall business operations?",
    {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'Economist'}}
---
{'Economist': {'messages': [HumanMessage(content="Changes in government policies regarding tariffs on imported goods can have significant impacts on supply chains and overall business operations. Here are several ways in which such changes might affect your business:\n\n1. **Cost of Goods Sold (COGS):** An increase in tariffs on imported goods usually means that the cost of those goods will rise. If your business relies on imported components or products, this could lead to an increase in your COGS. Conversely, a reduction in tariffs could lower your COGS.\n\n2. **Sourcing and Procurement:** Changes in tariffs might necessitate a reevaluation of your sourcing strategy. If tariffs make importing goods from a particular country cost-prohibitive, you may need to find alternative suppliers or consider reshoring some of your supply chain operations.\n\n3. **Pricing and Profit Margins:** Increased costs due to higher tariffs may lead to a need to rai

## Human Resources Team

In [10]:
import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import functools


# Research team graph state
class HumanResourcesTeamState(TypedDict):
    # A message is added after each team member finishes
    messages: Annotated[List[BaseMessage], operator.add]
    # The team members are tracked so they are aware of
    # the others' skill-sets
    team_members: List[str]
    # Used to route work. The supervisor calls a function
    # that will update this every time it makes a decision
    next: str


llm = ChatOpenAI(model="gpt-4-1106-preview")

recruitment_specialist_agent = create_agent(
    llm,
    recruitment_specialist_tools,
    "You are a Recruitment Specialist responsible for sourcing, screening, and selecting qualified candidates to fill job vacancies within an organization, ensuring a smooth and efficient hiring process." + prevent_endless_loops_template,
)
recruitment_specialist_node = functools.partial(agent_node, agent=recruitment_specialist_agent, name="Recruitment_Specialist")

payroll_and_personnel_agent = create_agent(
    llm,
    payroll_and_personnel_tools,
    "You are responsible for managing payroll processes and handling personnel-related matters within an organization, ensuring accurate and timely compensation for employees while maintaining compliance with regulations and policies." + prevent_endless_loops_template,
)
payroll_and_personnel_node = functools.partial(agent_node, agent=payroll_and_personnel_agent, name="Payroll_and_Personnel")

supervisor_agent = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  Recruitment_Specialist, Payroll_and_Personnel. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH. Do not call a tool more than once."
    "Do not call another tool if the tool returns results.",
    ["Recruitment_Specialist", "Payroll_and_Personnel"],
)

Human Resources Team Graph

In [11]:
human_resources_graph = StateGraph(HumanResourcesTeamState)
human_resources_graph.add_node("Recruitment_Specialist", recruitment_specialist_node)
human_resources_graph.add_node("Payroll_and_Personnel", payroll_and_personnel_node)
human_resources_graph.add_node("supervisor", supervisor_agent)

# Define the control flow
human_resources_graph.add_edge("Recruitment_Specialist", "supervisor")
human_resources_graph.add_edge("Payroll_and_Personnel", "supervisor")

human_resources_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Recruitment_Specialist": "Recruitment_Specialist",
        "Payroll_and_Personnel": "Payroll_and_Personnel",
        "FINISH": END
    },
)


human_resources_graph.set_entry_point("supervisor")
chain = human_resources_graph.compile()


# The following functions interoperate between the top level graph state
# and the state of the research sub-graph
# this makes it so that the states of each graph don't get intermixed
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
    }
    return results


human_resources_chain = enter_chain | chain

In [11]:
for s in human_resources_chain.stream(
"We're expanding our engineering team. How would you tailor our recruitment strategy to attract top talent in this competitive field?",
    {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'Recruitment_Specialist'}}
---
{'Recruitment_Specialist': {'messages': [HumanMessage(content="To attract top talent in the competitive field of engineering, a tailored recruitment strategy should include the following key elements:\n\n1. **Employer Branding:**\n   - Highlight the company's culture, mission, and values.\n   - Showcase innovative projects and technologies the team is working on.\n   - Share testimonials and stories from current engineers about their experiences and growth within the company.\n\n2. **Competitive Compensation and Benefits:**\n   - Research industry standards to offer competitive salaries.\n   - Provide a comprehensive benefits package that includes health, retirement savings plans, and perks like flexible work hours or remote work options.\n   - Consider offering sign-on bonuses or performance incentives.\n\n3. **Outreach and Sourcing:**\n   - Utilize professional networks like LinkedIn to identify and reach out to passive candidate

## Help Desk Team

In [12]:
import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import functools


# Research team graph state
class HelpDeskTeamState(TypedDict):
    # A message is added after each team member finishes
    messages: Annotated[List[BaseMessage], operator.add]
    # The team members are tracked so they are aware of
    # the others' skill-sets
    team_members: List[str]
    # Used to route work. The supervisor calls a function
    # that will update this every time it makes a decision
    next: str


llm = ChatOpenAI(model="gpt-4-1106-preview")

network_malfunction_specialist_agent = create_agent(
    llm,
    network_malfunction_specialist_tools,
    "You are a Network Malfunction Specialist tasked with diagnosing and resolving issues related to network infrastructure, such as connectivity problems, security breaches, or performance issues, to ensure smooth operations within an organization." + prevent_endless_loops_template,
)
network_malfunction_specialist_node = functools.partial(agent_node, agent=network_malfunction_specialist_agent, name="Network_Malfunction_Specialist")

mobile_banking_malfunction_specialist_agent = create_agent(
    llm,
    mobile_banking_malfunction_specialist_tools,
    "You are a Mobile Banking Malfunction Specialist responsible for troubleshooting and resolving technical issues encountered by users of mobile banking applications, ensuring a seamless and secure banking experience on mobile devices." + prevent_endless_loops_template,
)
mobile_banking_malfunction_specialist_node = functools.partial(agent_node, agent=mobile_banking_malfunction_specialist_agent, name="Mobile_Banking_Malfunction_Specialist")

transaction_error_specialist_agent = create_agent(
    llm,
    transaction_error_specialist_tools,
    "You are a Transaction Error Specialist tasked with investigating and rectifying errors or discrepancies in financial transactions, such as incorrect charges, failed payments, or account discrepancies, to maintain the integrity of financial records and ensure customer satisfaction." + prevent_endless_loops_template,
)
transaction_error_specialist_node = functools.partial(agent_node, agent=transaction_error_specialist_agent, name="Transaction_Error_Specialist")

customer_satisfaction_specialist_agent = create_agent(
    llm,
    customer_satisfaction_specialist_tools,
    "You are a Customer Satisfaction Specialist focused on monitoring customer feedback, addressing inquiries or complaints, and implementing strategies to enhance the overall customer experience and satisfaction with products or services offered by an organization." + prevent_endless_loops_template,
)
customer_satisfaction_specialist_node = functools.partial(agent_node, agent=customer_satisfaction_specialist_agent, name="Customer_Satisfaction_Specialist")


supervisor_agent = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  Network_Malfunction_Specialist, Mobile_Banking_Malfunction_Specialist, Transaction_Error_Specialist, Customer_Satisfaction_Specialist. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH. Do not call a tool more than once."
    "Do not call another tool if the tool returns results.",
    ["Network_Malfunction_Specialist", "Mobile_Banking_Malfunction_Specialist" , "Transaction_Error_Specialist" , "Customer_Satisfaction_Specialist"],
)

Help Desk Team Graph

In [13]:
help_desk_graph = StateGraph(HelpDeskTeamState)
help_desk_graph.add_node("Network_Malfunction_Specialist", network_malfunction_specialist_node)
help_desk_graph.add_node("Mobile_Banking_Malfunction_Specialist", mobile_banking_malfunction_specialist_node)
help_desk_graph.add_node("Transaction_Error_Specialist", transaction_error_specialist_node)
help_desk_graph.add_node("Customer_Satisfaction_Specialist", customer_satisfaction_specialist_node)
help_desk_graph.add_node("supervisor", supervisor_agent)

# Define the control flow
help_desk_graph.add_edge("Network_Malfunction_Specialist", "supervisor")
help_desk_graph.add_edge("Mobile_Banking_Malfunction_Specialist", "supervisor")
help_desk_graph.add_edge("Transaction_Error_Specialist", "supervisor")
help_desk_graph.add_edge("Customer_Satisfaction_Specialist", "supervisor")

help_desk_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Network_Malfunction_Specialist": "Network_Malfunction_Specialist",
        "Mobile_Banking_Malfunction_Specialist": "Mobile_Banking_Malfunction_Specialist",
        "Transaction_Error_Specialist": "Transaction_Error_Specialist",
        "Customer_Satisfaction_Specialist": "Customer_Satisfaction_Specialist",
        "FINISH": END
    },
)


help_desk_graph.set_entry_point("supervisor")
chain = help_desk_graph.compile()


# The following functions interoperate between the top level graph state
# and the state of the research sub-graph
# this makes it so that the states of each graph don't get intermixed
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
    }
    return results


help_desk_chain = enter_chain | chain

In [14]:
for s in help_desk_chain.stream(
    "How do you prioritize and troubleshoot network issues based on their severity and impact on operations?", 
    {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'Network_Malfunction_Specialist'}}
---
{'Network_Malfunction_Specialist': {'messages': [HumanMessage(content='Prioritizing and troubleshooting network issues effectively requires an understanding of the business impact, the severity of the issue, and the resources available to address the problem. Here’s a general approach to handling network issues:\n\n1. **Initial Assessment:**\n   - **Identify the scope of the issue:** Determine whether the problem is affecting a single user, a department, an entire office, or multiple locations.\n   - **Determine the impact on business operations:** Consider how critical the affected services are to the daily operations of the business.\n   - **Gather initial information:** Collect error messages, user reports, and any other relevant information.\n\n2. **Prioritization:**\n   - **Severity:** Issues that cause widespread outages or affect critical systems should be given the highest priority.\n   - **Urgency:** Consider deadl

In [15]:
for s in help_desk_chain.stream(
    "How do you address customer concerns regarding mobile banking app glitches or usability issues?", 
    {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'Mobile_Banking_Malfunction_Specialist'}}
---
{'Mobile_Banking_Malfunction_Specialist': {'messages': [HumanMessage(content="Addressing customer concerns regarding mobile banking app glitches or usability issues involves a multi-step process to ensure that the customer feels heard, supported, and satisfied with the resolution. Here is a general approach to manage such concerns effectively:\n\n1. **Listen and Understand:**\n   - Actively listen to the customer’s issue without interrupting.\n   - Ask clarifying questions to fully understand the problem.\n   - Show empathy and acknowledge the inconvenience caused by the glitch or issue.\n\n2. **Gather Information:**\n   - Collect necessary details such as the customer's device type, operating system version, app version, and specific actions leading to the issue.\n   - Request screenshots or error messages if necessary.\n\n3. **Initial Troubleshooting:**\n   - Guide the customer through basic troubleshooting steps, 

In [16]:
for s in help_desk_chain.stream(
    "What strategies do you employ to identify and rectify transaction errors promptly and accurately?",
    {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'Transaction_Error_Specialist'}}
---
{'Transaction_Error_Specialist': {'messages': [HumanMessage(content='As a Transaction Error Specialist, identifying and rectifying transaction errors promptly and accurately involves a systematic approach that includes several key strategies:\n\n1. **Error Detection Techniques**:\n   - Automated Alerts: Setting up automated systems that flag unusual activity or inconsistencies in transaction records.\n   - Reconciliation Procedures: Regularly comparing transaction records against account balances to ensure they match.\n   - Data Analysis: Utilizing data analytics to identify patterns that could indicate errors, such as duplicate transactions or outliers.\n\n2. **Investigation Procedures**:\n   - Transaction Auditing: Reviewing the details of transactions to verify their accuracy.\n   - Documentation Review: Examining related documentation such as receipts, invoices, or contracts for discrepancies.\n   - Communication: Contact

In [ ]:
for s in help_desk_chain.stream(
    "How do you handle challenging customer interactions to ensure a positive resolution and maintain overall customer satisfaction levels?",
    {"recursion_limit": 100}
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'Customer_Satisfaction_Specialist'}}
---


## Add Layers

In [14]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI


llm = ChatOpenAI(model="gpt-4-1106-preview")

supervisor_node = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following teams: {team_members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH. Do not call a tool more than once."
    "Do not call another tool if the tool returns results.",
    ["Research_Team", "Human_Resources_Team", "Help_Desk_Team"],
)

In [15]:
# Top-level graph state
class State(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    next: str


def get_last_message(state: State) -> str:
    return state["messages"][-1].content


def join_graph(response: dict):
    return {"messages": [response["messages"][-1]]}


# Define the graph.
super_graph = StateGraph(State)
# First add the nodes, which will do the work
super_graph.add_node("Research_Team", get_last_message | research_chain | join_graph)
super_graph.add_node(
    "Human_Resources_Team", get_last_message | human_resources_chain | join_graph
)
super_graph.add_node(
    "Help_Desk_Team", get_last_message | help_desk_chain | join_graph
)
super_graph.add_node("supervisor", supervisor_node)

# Define the graph connections, which controls how the logic
# propagates through the program
super_graph.add_edge("Research_Team", "supervisor")
super_graph.add_edge("Human_Resources_Team", "supervisor")
super_graph.add_edge("Help_Desk_Team", "supervisor")
super_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Research_Team": "Research_Team",
        "Human_Resources_Team": "Human_Resources_Team",
        "Help_Desk_Team": "Help_Desk_Team",
        "FINISH": END,
    },
)
super_graph.set_entry_point("supervisor")
super_graph = super_graph.compile()

In [16]:
for s in super_graph.stream(
    {
        "messages": [
            HumanMessage(
                content="A customer just informed us that he cannot login our mobile application and he sees an error message that says \"Authorization failed: Incorrect password\". Can you help him by listing out potential solutions?"
            )
        ],
    },
    {"recursion_limit": 150},
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'Help_Desk_Team'}}
---
{'Help_Desk_Team': {'messages': [HumanMessage(content='Certainly! If a customer is seeing an "Authorization failed: Incorrect password" error message when attempting to log in to a mobile banking application, here are some potential solutions to troubleshoot and resolve the issue:\n\n1. **Check Caps Lock and Typing Errors:**\n   - Ensure that the Caps Lock key is not enabled on the device\'s keyboard.\n   - Double-check that the password is being typed correctly, paying close attention to any characters that can be easily mistaken for others (e.g., O vs. 0, I vs. l).\n\n2. **Reset Password:**\n   - Use the "Forgot Password" or "Reset Password" feature typically found on the login screen to reset the password. This process often involves sending a secure link to the registered email address or a code to the registered phone number for verification.\n\n3. **Update Mobile Banking App:**\n   - Check if there is an update available for the mobi

KeyboardInterrupt: 

In [15]:
for s in super_graph.stream(
    {
        "messages": [
            HumanMessage(
                content="How can we improve our recruitment process to attract and retain top talent, while ensuring diversity and inclusion are prioritized throughout the hiring process?"
            )
        ],
    },
    {"recursion_limit": 150},
):
    if "__end__" not in s:
        print(s)
        print("---")

{'supervisor': {'next': 'Research_Team'}}
---
{'Research_Team': {'messages': [HumanMessage(content="Improving the recruitment process to attract and retain top talent, while ensuring diversity and inclusion, can involve several strategic actions and policy implementations. Here are steps to consider that can help achieve these goals:\n\n1. **Revise Job Descriptions:**\n   - Use inclusive language that is gender-neutral and free from bias.\n   - Highlight your company’s commitment to diversity and inclusion.\n   - Clearly define the role, responsibilities, and career growth opportunities.\n\n2. **Widen Recruitment Channels:**\n   - Partner with diverse professional organizations and universities.\n   - Utilize social media and job platforms that cater to a diverse audience.\n   - Encourage employee referrals while emphasizing the company’s diversity goals.\n\n3. **Implement Blind Hiring Practices:**\n   - Remove names and other demographic information from resumes.\n   - Use skills-base

KeyboardInterrupt: 